In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,TensorDataset
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.is_available())

cuda
True


In [2]:
def read_bci_data():
    S4b_train = np.load('S4b_train.npz')
    X11b_train = np.load('X11b_train.npz')
    S4b_test = np.load('S4b_test.npz')
    X11b_test = np.load('X11b_test.npz')

    train_data = np.concatenate((S4b_train['signal'], X11b_train['signal']), axis=0)
    train_label = np.concatenate((S4b_train['label'], X11b_train['label']), axis=0)
    test_data = np.concatenate((S4b_test['signal'], X11b_test['signal']), axis=0)
    test_label = np.concatenate((S4b_test['label'], X11b_test['label']), axis=0)


    train_label = train_label - 1
    test_label = test_label -1
    train_data = np.transpose(np.expand_dims(train_data, axis=1), (0, 1, 3, 2))
    test_data = np.transpose(np.expand_dims(test_data, axis=1), (0, 1, 3, 2))
   

    mask = np.where(np.isnan(train_data))
    train_data[mask] = np.nanmean(train_data)

    mask = np.where(np.isnan(test_data))
    test_data[mask] = np.nanmean(test_data)

   

    return train_data, train_label, test_data, test_label

In [3]:
train_data, train_label, test_data, test_label = read_bci_data()
dataset = TensorDataset(torch.from_numpy(train_data),torch.from_numpy(train_label))
loader_train = DataLoader(dataset,batch_size=256,shuffle=True,num_workers=4)
print(loader_train)
dataset = TensorDataset(torch.from_numpy(test_data),torch.from_numpy(test_label))
loader_test = DataLoader(dataset,batch_size=256,shuffle=False,num_workers=4)


In [10]:
class EEGNet(nn.Module):
    def __init__(self,activation=nn.ELU()):
        super(EEGNet,self).__init__()
        self.firstconv=nn.Sequential(
            nn.Conv2d(1,16,kernel_size=(1,51),stride=(1,1),padding=(0,25),bias=False),
            nn.BatchNorm2d(16,eps=1e-5,momentum=0.1,affine=True,track_running_stats=True)
        )
        self.depthwiseConv=nn.Sequential(
            nn.Conv2d(16,32,kernel_size=(2,1),stride=(1,1),groups=16,bias=False),
            nn.BatchNorm2d(32,eps=1e-5,momentum=0.1,affine=True,track_running_stats=True),
            activation,
            nn.AvgPool2d(kernel_size=(1,4),stride=(1,4),padding=0),
            nn.Dropout(p=0.25)
        )
        self.seperableConv=nn.Sequential(
            nn.Conv2d(32,32,kernel_size=(1,15),stride=(1,1),padding=(0,7),bias=False),
            nn.BatchNorm2d(32,eps=1e-5,momentum=0.1,affine=True,track_running_stats=True),
            activation,
            nn.AvgPool2d(kernel_size=(1,8),stride=(1,8),padding=0),
            nn.Dropout(p=0.25)
        )
        self.classify=nn.Linear(736,2)
    def forward(self,inputs):
        output = self.firstconv(inputs)
        output = self.depthwiseConv(output)
        output = self.seperableConv(output)
        output = torch.flatten(output, start_dim=1)
        return self.classify(output)
    
    def calculate_accuracy(self, pred_y, label):
        correct = pred_y.max(dim=1)[1].eq(label).sum().item()
        total = len(label)
        accuracy = (correct / total) * 100
        return accuracy
    
    
    def train_and_test(self, loader_train,loader_test):
        batch_size= 64        
        lr = 1e-2        
        activations={nn.ReLU(),nn.LeakyReLU(),nn.ELU()}
        loss = torch.nn.CrossEntropyLoss()
        train_acc = []
        
        for act in activations:
            model=EEGNet(act)
            model.to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            for epoch in range(150):
                model.train()
                all_loss = 0
                print(epoch)
                for idx,(data,label) in enumerate(loader_train):
                    data=data.to(device,dtype=torch.float)
                    label=label.to(device,dtype=torch.long)
                    pred_y = model(data)
                    mono_loss = loss(pred_y, label)
                    all_loss += mono_loss
                    accuracy = self.calculate_accuracy(pred_y, label)
                    if epoch % 5 == 0:
                        print(f"Epoch: {epoch}, Loss: {all_loss/len(loader_train.dataset)}, Accuracy: {accuracy:.2f}%")
                    optimizer.zero_grad()
                    mono_loss.backward() 
                    optimizer.step()    
            print("Training finished.")
        print('start testing:')
        model.eval()
        for index, data in enumerate(test_data):
            predict = model(data)
            loss = loss(test_label, predict)
            accuracy = self.calculate_accuracy(pred_y, test_label)
        print(f"Loss: {np.mean(loss)}, Accuracy: {accuracy:.2f}%")
        print('testing finished')
    
    


In [ ]:
neural_network = EEGNet()
neural_network.train_and_test(loader_train,loader_test)

0
Epoch: 0, Loss: 0.0007093040621839464, Accuracy: 43.75%
Epoch: 0, Loss: 0.0013329839566722512, Accuracy: 60.16%
Epoch: 0, Loss: 0.0018630385166034102, Accuracy: 73.05%
Epoch: 0, Loss: 0.0024113778490573168, Accuracy: 74.22%
Epoch: 0, Loss: 0.0033081313595175743, Accuracy: 69.64%
1
2
3
4
5
Epoch: 5, Loss: 0.00045142779708839953, Accuracy: 73.44%
Epoch: 5, Loss: 0.0008849237347021699, Accuracy: 74.22%
Epoch: 5, Loss: 0.001379515277221799, Accuracy: 70.31%
Epoch: 5, Loss: 0.0018725770059973001, Accuracy: 74.61%
Epoch: 5, Loss: 0.002175818430259824, Accuracy: 89.29%
6
7
8
9
10
Epoch: 10, Loss: 0.00043485360220074654, Accuracy: 74.22%
Epoch: 10, Loss: 0.0008680610335431993, Accuracy: 77.34%
Epoch: 10, Loss: 0.001254543662071228, Accuracy: 78.52%
Epoch: 10, Loss: 0.0017235722625628114, Accuracy: 73.83%
Epoch: 10, Loss: 0.0021188193932175636, Accuracy: 75.00%
11
12
13
14
15
Epoch: 15, Loss: 0.0003857033443637192, Accuracy: 80.47%
Epoch: 15, Loss: 0.0008101363200694323, Accuracy: 75.78%
Epoc

In [ ]:
activations={nn.ReLU(),nn.LeakyReLU(),nn.ELU()}
for activation in activations:
    print (activation)